<a href="https://colab.research.google.com/github/chefdarek/DS-Unit-2-Regression-2/blob/master/module1-log-linear-regression/LogRegression_Caterpillar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [4]:
!unzip caterpillar-tube-pricing.zip

Archive:  caterpillar-tube-pricing.zip
  inflating: sample_submission.csv   
  inflating: data.zip                


In [5]:
!unzip data.zip

Archive:  data.zip
   creating: competition_data/
  inflating: competition_data/bill_of_materials.csv  
  inflating: competition_data/comp_adaptor.csv  
  inflating: competition_data/comp_boss.csv  
  inflating: competition_data/comp_elbow.csv  
  inflating: competition_data/comp_float.csv  
  inflating: competition_data/comp_hfl.csv  
  inflating: competition_data/comp_nut.csv  
  inflating: competition_data/comp_other.csv  
  inflating: competition_data/comp_sleeve.csv  
  inflating: competition_data/comp_straight.csv  
  inflating: competition_data/comp_tee.csv  
  inflating: competition_data/comp_threaded.csv  
  inflating: competition_data/components.csv  
  inflating: competition_data/specs.csv  
  inflating: competition_data/test_set.csv  
  inflating: competition_data/train_set.csv  
  inflating: competition_data/tube.csv  
  inflating: competition_data/tube_end_form.csv  
  inflating: competition_data/type_component.csv  
  inflating: competition_data/type_connection.csv  
  i

In [6]:
from glob import glob
import pandas as pd

for path in glob('competition_data/*.csv'):
    df = pd.read_csv(path)
    print(path, df.shape)

competition_data/bill_of_materials.csv (21198, 17)
competition_data/comp_float.csv (16, 7)
competition_data/comp_adaptor.csv (25, 20)
competition_data/comp_hfl.csv (6, 9)
competition_data/comp_other.csv (1001, 3)
competition_data/comp_threaded.csv (194, 32)
competition_data/type_end_form.csv (8, 2)
competition_data/specs.csv (21198, 11)
competition_data/comp_boss.csv (147, 15)
competition_data/test_set.csv (30235, 8)
competition_data/type_component.csv (29, 2)
competition_data/comp_tee.csv (4, 14)
competition_data/tube_end_form.csv (27, 2)
competition_data/comp_straight.csv (361, 12)
competition_data/train_set.csv (30213, 8)
competition_data/comp_elbow.csv (178, 16)
competition_data/type_connection.csv (14, 2)
competition_data/components.csv (2048, 3)
competition_data/comp_nut.csv (65, 11)
competition_data/comp_sleeve.csv (50, 10)
competition_data/tube.csv (21198, 16)


In [7]:
!pip install category_encoders

     |████████████████████████████████| 92kB 5.6MB/s 


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
import category_encoders as ce
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error



In [ ]:

def rmse (y_true, y_pred):
    return  np.sqrt(mean_squared_error(y_true, y_pred))
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [ ]:
bill_of_materials = pd.read_csv('competition_data/bill_of_materials.csv')
comp_adaptor = pd.read_csv('competition_data/comp_adaptor.csv')
comp_boss = pd.read_csv('competition_data/comp_boss.csv')
comp_elbow = pd.read_csv('competition_data/comp_elbow.csv')
comp_float = pd.read_csv('competition_data/comp_float.csv')
comp_hf1 = pd.read_csv('competition_data/comp_hfl.csv')
comp_nut = pd.read_csv('competition_data/comp_nut.csv')
comp_other = pd.read_csv('competition_data/comp_other.csv')
comp_sleeve = pd.read_csv('competition_data/comp_sleeve.csv')
comp_straight = pd.read_csv('competition_data/comp_straight.csv')
comp_tee = pd.read_csv('competition_data/comp_tee.csv')
comp_threaded = pd.read_csv('competition_data/comp_threaded.csv')
components = pd.read_csv('competition_data/components.csv')
specs = pd.read_csv('competition_data/specs.csv')
tube_end_form = pd.read_csv('competition_data/tube_end_form.csv')
type_component = pd.read_csv('competition_data/type_component.csv')
type_connection = pd.read_csv('competition_data/type_connection.csv')
type_end_form = pd.read_csv('competition_data/type_end_form.csv')
trainval = pd.read_csv('competition_data/train_set.csv')
test = pd.read_csv('competition_data/test_set.csv')
tube = pd.read_csv('competition_data/tube.csv')

In [49]:
for path in glob('competition_data/*.csv'):
    df = pd.read_csv(path)
    shared_columns = set(df.columns) & set(train.columns)
    if shared_columns:
        print(path, df.shape)
        print(df.columns.tolist(), '\n')

competition_data/bill_of_materials.csv (21198, 17)
['tube_assembly_id', 'component_id_1', 'quantity_1', 'component_id_2', 'quantity_2', 'component_id_3', 'quantity_3', 'component_id_4', 'quantity_4', 'component_id_5', 'quantity_5', 'component_id_6', 'quantity_6', 'component_id_7', 'quantity_7', 'component_id_8', 'quantity_8'] 

competition_data/specs.csv (21198, 11)
['tube_assembly_id', 'spec1', 'spec2', 'spec3', 'spec4', 'spec5', 'spec6', 'spec7', 'spec8', 'spec9', 'spec10'] 

competition_data/test_set.csv (30235, 8)
['id', 'tube_assembly_id', 'supplier', 'quote_date', 'annual_usage', 'min_order_quantity', 'bracket_pricing', 'quantity'] 

competition_data/train_set.csv (30213, 8)
['tube_assembly_id', 'supplier', 'quote_date', 'annual_usage', 'min_order_quantity', 'bracket_pricing', 'quantity', 'cost'] 

competition_data/tube.csv (21198, 16)
['tube_assembly_id', 'material_id', 'diameter', 'wall', 'length', 'num_bends', 'bend_radius', 'end_a_1x', 'end_a_2x', 'end_x_1x', 'end_x_2x', 'end

In [ ]:
trainval['quote_date'] = pd.to_datetime(trainval['quote_date'], infer_datetime_format=True)
test['quote_date'] = pd.to_datetime(test['quote_date'], infer_datetime_format=True)

In [53]:
trainval_tube_assemblies = trainval['tube_assembly_id'].unique()
test_tube_assemblies = test['tube_assembly_id'].unique()
len(trainval_tube_assemblies), len(test_tube_assemblies)

(8855, 8856)

In [ ]:
from sklearn.model_selection import train_test_split
train_tube_assemblies, val_tube_assemblies = train_test_split(
    trainval_tube_assemblies, random_state=42)

In [55]:
train = trainval[trainval.tube_assembly_id.isin(train_tube_assemblies)]
val = trainval[trainval.tube_assembly_id.isin(val_tube_assemblies)]
train.shape, val.shape, trainval.shape

((22628, 8), (7585, 8), (30213, 8))

In [ ]:
test = test.drop('id', axis=1)

In [60]:
target = 'cost'
y_train = train[target]
y_val = val[target]
y_pred = np.full_like(y_val, fill_value=y_train.mean())
print('Validation RMSLE, Mean Baseline:', rmsle(y_val, y_pred))

Validation RMSLE, Mean Baseline: 0.9418101276064408


###Log the target 

In [ ]:
y_train_log = np.log1p(y_train)
Y_val_log = np.log1p(y_val)

In [ ]:
for path in glob('competition_data/*.csv'):
    df = pd.read_csv(path)
    shared_columns = set(df.columns) & set(components.columns)
    if shared_columns:
        print(path, df.shape)
        print(df.columns.tolist(), '\n')

In [ ]:
for path in glob('competition_data/*.csv'):
    df = pd.read_csv(path)
    shared_columns = set(df.columns) & set(tube.columns)
    if shared_columns:
        print(path, df.shape)
        print(df.columns.tolist(), '\n')
        #print(df.head())

In [51]:
trainval.describe(include='object')

,tube_assembly_id,supplier,quote_date,bracket_pricing
count,30213,30213,30213,30213
unique,8855,57,1781,2
top,TA-04788,S-0066,2013-10-01,Yes
freq,14,20553,2877,26283


In [46]:
test.describe(include='object')

,tube_assembly_id,supplier,quote_date,bracket_pricing
count,30235,30235,30235,30235
unique,8856,56,1778,2
top,TA-04027,S-0066,2013-09-01,Yes
freq,12,20547,2992,26279


In [ ]:
features  train

In [ ]:
bill_of_materials.describe(include="all")

In [ ]:
specs.describe(include="all")

In [ ]:
tube.describe(include="all")

In [ ]:
components.describe(include="all")

In [ ]:
comp_sleeve.describe(include="all")

In [ ]:
type_end_form.head()

In [ ]:
type_end_form.head()

In [ ]:
type_connection.describe()

In [ ]:
tube_end_form.describe()

In [ ]:
train = trainval[trainval.tube_assembly_id.isin(train_tube_assemblies)]
val = trainval[trainval.tube_assembly_id.isin(val_tube_assemblies)]
train.shape, val.shape, trainval.shape

In [ ]:
def wrangle(X):
    X = X.copy()
    
    # Engineer date features
    X['quote_date'] = pd.to_datetime(X['quote_date'], infer_datetime_format=True)
    X['quote_date_year'] = X['quote_date'].dt.year
    X['quote_date_month'] = X['quote_date'].dt.month
    X = X.drop(columns='quote_date')
    
    # Merge tube data
    tube = pd.read_csv('competition_data/tube.csv')
    X = X.merge(tube, how='left')
    
    # Drop tube_assembly_id because our goal is to predict unknown assemblies
    #The TEST data and train had no matching assembly id's 
    X = X.drop(columns='tube_assembly_id')
    
    return X


train_wrangled = wrangle(train)
#test has to match in columns 
test_wrangled = wrangle(test)
val_wrangled = wrangle(val)

In [112]:
test_wrangled.head()

,supplier,annual_usage,min_order_quantity,bracket_pricing,quantity,quote_date_year,quote_date_month,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other
0,S-0066,0,0,Yes,1,2013,6,SP-0035,12.7,1.65,164.0,5,38.1,N,N,N,N,EF-003,EF-003,0,0,0
1,S-0066,0,0,Yes,2,2013,6,SP-0035,12.7,1.65,164.0,5,38.1,N,N,N,N,EF-003,EF-003,0,0,0
2,S-0066,0,0,Yes,5,2013,6,SP-0035,12.7,1.65,164.0,5,38.1,N,N,N,N,EF-003,EF-003,0,0,0
3,S-0066,0,0,Yes,10,2013,6,SP-0035,12.7,1.65,164.0,5,38.1,N,N,N,N,EF-003,EF-003,0,0,0
4,S-0066,0,0,Yes,25,2013,6,SP-0035,12.7,1.65,164.0,5,38.1,N,N,N,N,EF-003,EF-003,0,0,0


In [63]:
features = train_wrangled.columns.drop('cost')
print('Features:',features.tolist())

Features: ['supplier', 'annual_usage', 'min_order_quantity', 'bracket_pricing', 'quantity', 'quote_date_year', 'quote_date_month', 'material_id', 'diameter', 'wall', 'length', 'num_bends', 'bend_radius', 'end_a_1x', 'end_a_2x', 'end_x_1x', 'end_x_2x', 'end_a', 'end_x', 'num_boss', 'num_bracket', 'other']


In [ ]:

X_train = train_wrangled[features]
X_val = val_wrangled[features]
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

In [89]:
y_train.shape

(22628,)

In [86]:
X_train.shape
#print(y_val_log.shape)

(22628, 22)

In [ ]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(), 
    RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    
)

pipeline.fit(X_train, y_train_log)
y_pred_log = pipeline.predict(X_val)




Only using the RMSE because true and pred are already log

In [81]:
rmse(y_val_log, y_pred_log)

0.3071425826783341

Test Set Prediction

In [ ]:
X_test = test_wrangled[features]

In [ ]:
y_pred_log = pipeline.predict(X_test)

In [120]:
y_pred_log.shape

(30235,)

In [126]:
test_set = pd.read_csv('competition_data/test_set.csv')
test_set.shape

(30235, 8)

In [ ]:
test_set = pd.DataFrame(test_set.id)

In [ ]:
y_pred = np.expm1(y_pred_log)
submission = test_set.copy()
submission['cost'] = y_pred
submission.to_csv('DareksubmissionCatepillar-01.csv', index=False)

In [132]:
submission.head()

,id,cost
0,1,23.372078
1,2,13.786769
2,3,7.868861
3,4,5.840275
4,5,4.675781


In [ ]:
#933 real score DareksubmissionCatepillar-01(2).csv
#just now
#0 seconds
#0 seconds
Kaggle Score
#0.30198